# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

In [4]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

We would like to use a proportions z-test to see whether black-sounding and white-sounding groups differ in their callback rate. The Central Limit Theorem does not apply. The null hypothesis is that the proportions of callback rates are equal for the two races, while the alternative hypothesis is that there is a difference in the proportions of callback rates between the two races. 

In [29]:
#Separate data into white and black groups
w = data[data.race=='w']
b = data[data.race=='b']

#Calculate proportions of callback rates for both groups and calculate difference
w_prop = np.sum(w.call == 1) / len(w)
b_prop = np.sum(b.call == 1) / len(b)
emp_diff_prop = w_prop - b_prop

#Generate 1000 bootstrap samples
diff_prop=np.empty(1000)

for i in range(1000):
    #Create two samples after bootstrapping and take difference in proportions of callback rates
    bs_sample = np.random.choice(data['call'], size=len(data['call']))
    sample_1 = bs_sample[:len(w)]
    sample_2 = bs_sample[len(w):]
    prop_1 = np.sum(sample_1 == 1) / len(sample_1)
    prop_2 = np.sum(sample_2 == 1) / len(sample_2)
    diff_prop[i] = prop_1 - prop_2

#Calculate confidence intervals and p-values for proportion difference
conf_int = np.percentile(diff_prop, [2.5, 97.5])
p = np.sum(diff_prop >= emp_diff_prop) / len(diff_prop)
print("bootstrap confidence interval:", conf_int)
print("bootstrap p-value:", p)

#Run proportions z-test on black and white groups
stat, p = proportions_ztest([np.sum(w.call == 1), np.sum(b.call == 1)], [len(w), len(b)])
print("z test statistic: %.3f, p-value: %.3f" %(stat, p))

bootstrap confidence interval: [-0.01520534  0.01560575]
bootstrap p-value: 0.0
z test statistic: 4.108, p-value: 0.000


The fact that we got a p-value of 0 for both our boostrap and frequentist approaches highly suggests that we can reject our null hypothesis that there is no difference in callback rates between whites and blacks. 
However, this does not suggest that race is THE single most important factor in callback success. It is important, but this might be because it is correlated with other variables that are more closely related to callback rates, or there might be other variables which are still more important. To account for these possibilities, we could run tests that would look at the strength of the relationships between other variables and callback rates, as well as the strength of the relationships between the variables themselves. 